<a href="https://colab.research.google.com/github/Polineska/SpeechRecognition/blob/main/vosk_analysisGolos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!sudo apt update && sudo apt install ffmpeg
!pip install jiwer -q

In [ ]:
!pip3 install vosk

In [ ]:
import sys
sys.executable
import os
import warnings
import jiwer
import torch
from datasets import load_dataset
import numpy as np

In [ ]:
#bond005/sberdevices_golos_10h_crowd_noised_2db
DATASET_ID = "bond005/sberdevices_golos_10h_crowd"
SAMPLES = 10

test_dataset = load_dataset(DATASET_ID, split=f"test[:{SAMPLES}]")



Generating train split:   0%|          | 0/7993 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/793 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9994 [00:00<?, ? examples/s]

In [ ]:
from vosk import Model, KaldiRecognizer
import wave
import json

In [ ]:
import os
import warnings
import vosk
import jiwer
import numpy as np
import pandas as pd

In [ ]:
import librosa

def speech_file_to_array_fn(batch):
    audio_file = batch["audio"].get("path")
    if audio_file is not None:
        try:
            speech_array, sampling_rate = librosa.load(audio_file, sr=16000)
            speech_array = speech_array.astype(np.int16)
            batch["speech"] = speech_array
        except Exception as e:
            print(f"Error loading file {audio_file}: {e}")
    return batch

# Предобработка датасета
removed_columns = set(test_dataset.column_names)
removed_columns -= {'transcription', 'speech'}
removed_columns = sorted(list(removed_columns))
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    test_dataset = test_dataset.map(
        speech_file_to_array_fn,
        remove_columns=removed_columns
    )

# Функция для распознавания речи с помощью Vosk
def infer_vosk(samples):
    transcripts = []
    model = Model(lang='ru')
    for i, sample in enumerate(samples):
        if "speech" in sample:
            audio = sample["speech"]
            rec = KaldiRecognizer(model, 16000)
            rec.SetMaxAlternatives(10)
            rec.SetPartialWords(True)

            # Распознавание речи
            rec.AcceptWaveform(audio.tobytes())
            result = rec.FinalResult()

            # Получение текста распознавания
            if isinstance(result, dict):
                alternatives = result.get_alternatives()
                if alternatives:
                    transcript_text = alternatives[0].get('text', '')
                    return transcripts

# Распознавание речи и вывод результатов
hypotheses = infer_vosk(test_dataset)
references = test_dataset["transcription"]


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(dict(hypothesis_small=hypotheses, reference_small=references))

In [ ]:
import re

def normalize_text(text):
    if not isinstance(text, str):
        text = str(text)

    # Удаление знаков пунктуации и преобразование в нижний регистр
    text = re.sub(r'[^\w\s]', '', text).lower()

    # Удаление множественных пробелов
    text = re.sub(r'\s+', ' ', text)

    return text.strip()
normalized_refs = [normalize_text(ref) for ref in test_dataset["transcription"]]
normalized_hyps = [normalize_text(hyp) for hyp in hypotheses]
# Вычисление WER с нормализованными транскрипциями
wer_score = jiwer.wer(normalized_refs, normalized_hyps)
print(f"\nWord Error Rate: {wer_score * 100:.2f}%")


Word Error Rate: 87.50%
